In [85]:
import cv2
import numpy as np
import os

In [86]:
sequence_folder = 'sequence/'
frame_files = sorted([f for f in os.listdir(sequence_folder) if f.endswith('.jpg') or f.endswith('.png')])
initial_bbox = (246, 208, 23, 23) 

initial_hypothesis = np.array([initial_bbox[1], initial_bbox[0], 
                                   initial_bbox[1] + initial_bbox[3], 
                                   initial_bbox[0] + initial_bbox[2]])
frames = (cv2.imread(os.path.join(sequence_folder, f)) for f in frame_files)
frame = list(frames)[0]

In [87]:
top, left, bottom, right = initial_hypothesis

In [88]:
object_mask = np.zeros(frame.shape[:2], dtype=bool)
object_mask[top:bottom, left:right] = True


In [89]:
padding = 20
surrounding_top = max(0, top - padding)
surrounding_left = max(0, left - padding)
surrounding_bottom = min(frame.shape[0], bottom + padding)
surrounding_right = min(frame.shape[1], right + padding)
surrounding_mask = np.zeros(frame.shape[:2], dtype=bool)
surrounding_mask[surrounding_top:surrounding_bottom, surrounding_left:surrounding_right] = True
surrounding_mask[top:bottom, left:right] = False  # Exclude the object region

In [ ]:
def compute_histogram(image, region_mask, bins):
    # histogram = np.zeros(bins)
    histogram = cv2.calcHist([image], [0], region_mask.astype(np.uint8), [bins], [0, 256]).flatten()
    print(histogram.shape)

    # for c in range(image.shape[-1]):  
    #     pixel_values = image[..., c][region_mask] 
    #     hist, _ = np.histogram(pixel_values, bins=bins, range=(0, 256))
    #     histogram += hist
    return histogram
    # histogram = cv2.calcHist([image], [0], region_mask.astype(np.uint8), [bins], [0, 256]).flatten()

In [91]:
bins = 250

In [92]:
H_O = compute_histogram(frame, object_mask, bins)
H_S = compute_histogram(frame, surrounding_mask, bins)


(250,)
(250,)


In [93]:
probabilities = np.zeros(frame.shape[:2])
probabilities.shape

(360, 640)

In [94]:

def calculateProbabilities(I_x, H_O, H_S):

    H_O_b = H_O[I_x] 
    H_S_b = H_S[I_x] 

    total_hist = H_O_b + H_S_b

    return H_O_b / total_hist


In [95]:
for y in range(frame.shape[0]):
    for x in range(frame.shape[1]):
            pixel_values = frame[y, x]
            bin_index = int((pixel_values.mean() / 256) * bins)  # Map pixel to bin
            
            if surrounding_mask[y, x] or object_mask[y, x]:
                probabilities[y, x] = calculateProbabilities(bin_index, H_O, H_S)
            else:
                probabilities[y, x] = 0.5



/var/folders/y9/2rxtmq5d4rs75j3fynzjqhxh0000gn/T/ipykernel_28784/2461442818.py:8: RuntimeWarning: invalid value encountered in scalar divide
  return H_O_b / total_hist


In [96]:
cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
# object_mask_display = (object_mask * 205).astype(np.uint8)
color_map = cv2.applyColorMap((probabilities * 255).astype(np.uint8), cv2.COLORMAP_RAINBOW)
vis_likelihood = cv2.addWeighted(frame, 0.1, color_map, 0.9, 0)
vis = np.hstack((frame, vis_likelihood))
cv2.imshow('Object Tracking', probabilities)
key = cv2.waitKey(0) & 0xFF
cv2.destroyAllWindows()

/var/folders/y9/2rxtmq5d4rs75j3fynzjqhxh0000gn/T/ipykernel_28784/2100857108.py:3: RuntimeWarning: invalid value encountered in cast
  color_map = cv2.applyColorMap((probabilities * 255).astype(np.uint8), cv2.COLORMAP_RAINBOW)
